In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from typing import Literal
import numpy as np

In [ ]:
# Load California Housing dataset
california = fetch_california_housing()
X, y = california.data, california.target

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Mini-Batch Gradient Descent
sgd_mini_batch = SGDRegressor(
    loss='squared_error',
    learning_rate='constant',
    eta0=0.005,  # Moderate learning rate
    max_iter=1,  # Single pass per fit call
    tol=None,
    shuffle=True,
    random_state=42
)

# Mini-batch settings
batch_size = 5
n_samples = X_train_scaled.shape[0]
n_iterations = 50

# Simulate mini-batch updates
for i in range(n_iterations):
    indices = np.random.choice(n_samples, batch_size, replace=False)
    X_batch = X_train_scaled[indices]
    y_batch = y_train[indices]
    sgd_mini_batch.partial_fit(X_batch, y_batch)


In [ ]:
# Predict and evaluate
y_pred_mini_batch = sgd_mini_batch.predict(X_test_scaled)
mse_mini_batch = mean_squared_error(y_test, y_pred_mini_batch)
print(f"Mini-Batch Gradient Descent MSE: {mse_mini_batch:.4f}")

In [ ]:
# Coefficients and intercept
print(f"Coefficients: {sgd_mini_batch.coef_}")
print(f"Intercept: {sgd_mini_batch.intercept_}")

# Mini-Batch Gradient Descent with Ridge and Lasso Regression

In [ ]:
# Custom Mini-Batch Gradient Descent
def mini_batch_sgd(
    X, 
    y, 
    batch_size=32, 
    penalty: Literal['l2', 'l1', 'elasticnet'] = 'l2', 
    alpha=0.01, 
    eta0=0.01, 
    max_iter=1000
):
    n_samples = X.shape[0]
    model = SGDRegressor(loss='squared_error',penalty=penalty, alpha=alpha, 
                         learning_rate='constant', eta0=eta0, max_iter=1, 
                         random_state=42, shuffle=False)
    
    for epoch in range(max_iter):
        indices = np.random.permutation(n_samples)
        for start_idx in range(0, n_samples, batch_size):
            batch_indices = indices[start_idx:start_idx + batch_size]
            X_batch = X[batch_indices]
            y_batch = y[batch_indices]
            model.partial_fit(X_batch, y_batch)
    return model

# Ridge Regression with Mini-Batch Gradient Descent
ridge_mini = mini_batch_sgd(X_train_scaled, y_train, batch_size=32, penalty='l2')
y_pred_ridge_mini = ridge_mini.predict(X_test_scaled)
ridge_mini_mse = mean_squared_error(y_test, y_pred_ridge_mini)
print(f"Ridge (Mini-Batch GD) MSE: {ridge_mini_mse:.4f}")

# Lasso Regression with Mini-Batch Gradient Descent
lasso_mini = mini_batch_sgd(X_train_scaled, y_train, batch_size=32, penalty='l1')
y_pred_lasso_mini = lasso_mini.predict(X_test_scaled)
lasso_mini_mse = mean_squared_error(y_test, y_pred_lasso_mini)
print(f"Lasso (Mini-Batch GD) MSE: {lasso_mini_mse:.4f}")